# CMAP EMAT

In [ ]:
import emat
emat.versions()

In [ ]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.INFO)

## Connecting to the Model

The interface for this model is located in the `cmap_emat.py`
module, which we will import into this notebook.  This file is extensively
documented in comments, and is a great starting point for new users
who want to edit this interface.

In [ ]:
import cmap_emat
fx = cmap_emat.CMAP_EMAT_Model()

In [ ]:
fx.db.read_scope('CMAP_Trip_2020') == fx.scope

In [ ]:
fx.invalidate_experiment_runs(
    "ENTIRE_NETWORK_Person_Trips_HO_Transit_Person_Trips == 0",
    "ENTIRE_NETWORK_Person_Trips_HW_Transit_Person_Trips == 0",
)

In [ ]:
results = fx.db.read_experiment_all(
    design_name='lhs', 
    only_with_measures=False, 
    scope_name=fx.scope.name, 
    with_run_ids=True,
)

In [ ]:
measures_1 = [
    'Regionwide_VMT',
    'Regionwide_Arterial_VMT',
    'Trip_Duration_Ratio_EJ_to_Global_HW',
    'ENTIRE_NETWORK_Transit_Share_HW_Transit_Share',
    'ENTIRE_NETWORK_Transit_Share_Overall_Transit_Share',
    'ENTIRE_NETWORK_Trip_Distance_HW_Trip_Average_Miles',
    'ENTIRE_NETWORK_Trip_Duration_HW_Trip_Average_Minutes',
    'mf44_amtime_2004_to_24', # Lombard to the Loop
]

measures_2 = [
    'Chicago_Expressway_VMT',
    'Chicago_Arterial_VMT',
    'EJ_Average_Trip_Time_HWej_Aut_Avg_Min',
    'ENTIRE_NETWORK_Trip_Duration_HW_Trip_Average_Minutes',
]

In [ ]:
from emat.analysis import feature_scores
feature_scores(fx.scope, results, measures=measures_1+measures_2)

## Review Core Model Experiments

In [ ]:
from emat.analysis import Visualizer

In [ ]:
vz = Visualizer(
    data=results, 
    scope=fx.scope,
)

### Lever Selectors

In [ ]:
vz.lever_selectors()

### Uncertainty Selectors

In [ ]:
vz.uncertainty_selectors()

### Measure Selectors (Curated)

In [ ]:
vz.selectors(measures_1+measures_2)

### SPLOM

In [ ]:
vz.splom(
    rows=measures_2,
    cols=[
        'highway_cap',
        'transit_fares',
        'expressway_toll',
    ],
)

### Two Way Visualizer

In [ ]:
tw = vz.two_way(x='highway_cap', y='Chicago_Expressway_VMT')
tw

### Three Way Visualizer

In [ ]:
v3 = vz.three_way(
    x = 'vmt_charge',
    y = 'fuel_cost',
    z = 'land_use',
)
v3

## Some Calculated Selections

In [ ]:
vz.new_selection(
    vz.data['Chicago_Expressway_VMT'].isna(),
    name='MissingResults',
)

In [ ]:
vz.new_selection(
    emat.Box("AltLandUse", allowed={'land_use':['alt1']}),
)

## Selected Feature Scores

In [ ]:
fx.feature_scores('lhs', measures=measures_1+measures_2)

In [ ]:
from emat.analysis import display_experiments

## Scatter Rows (Static)

In [ ]:
display_experiments(fx.scope, 'lhs', db=fx.db, rows=measures_1)

## Metamodel

In [ ]:
mm = emat.create_metamodel(
    scope=fx.scope,
    experiments=results[~results.Regionwide_Arterial_VMT.isna()].copy(),
    metamodel_id=999,
    db=None,
    include_measures=measures_1+measures_2,
)

In [ ]:
mm_design = mm.design_experiments(n_samples=10_000)

In [ ]:
mm_results = mm.run_experiments(mm_design)

In [ ]:
from emat.analysis import contrast_experiments

### Contrast Experiments

In [ ]:
contrast_experiments(
    fx.scope, 
    mm_results,
    results,
    rows=measures_1
)

### Interactive Visualizer

In [ ]:
mmvz = Visualizer(
    data=mm_results, 
    scope=fx.scope,
)

In [ ]:
mmvz.complete(measures=measures_1)

In [ ]:
mmvz.two_way()